In [10]:
import pandas as pd

In [11]:
#Load business file

df_b = pd.read_json("yelp_academic_dataset_business.json", lines=True)

ERROR! Session/line number was not unique in database. History logging moved to new session 186


In [12]:
#First five rows
df_b.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,1,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ..."
2,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,4720 Hawthorne Ave,Portland,OR,97214,45.511907,-122.613693,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Antiques, Fashion, Used, Vintage & Consignment...","{'Thursday': '11:0-18:0', 'Friday': '11:0-18:0..."
3,oaepsyvc0J17qwi8cfrOWg,Great Clips,2566 Enterprise Rd,Orange City,FL,32763,28.914482,-81.295979,3.0,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Beauty & Spas, Hair Salons",None
4,PE9uqAjdw0E4-8mjGl3wVA,Crossfit Terminus,1046 Memorial Dr SE,Atlanta,GA,30316,33.747027,-84.353424,4.0,14,1,"{'GoodForKids': 'False', 'BusinessParking': '{...","Gyms, Active Life, Interval Training Gyms, Fit...","{'Monday': '16:0-19:0', 'Tuesday': '16:0-19:0'..."


In [13]:
#Clean Business File
df_b = df_b[df_b['is_open']==1].drop(["address", "postal_code", "latitude", "longitude", "hours"], axis=1)

In [14]:
#List categories
df_explode = df_b.assign(categories = df_b.categories
                         .str.split(', ')).explode('categories')
df_explode.categories.value_counts()[:15]

Restaurants                  32022
Food                         20418
Shopping                     20309
Home Services                14795
Health & Medical             13630
Beauty & Spas                13569
Local Services               10677
Automotive                    8886
Event Planning & Services     7856
Nightlife                     7829
Active Life                   7675
Bars                          7008
Coffee & Tea                  5476
Hotels & Travel               5032
Sandwiches                    4795
Name: categories, dtype: int64

In [15]:
#List all categories with restaurant
df_explode[df_explode.categories.str.contains('Restaurants',
                      case=True,na=False)].categories.value_counts()

Restaurants           32022
Pop-Up Restaurants       48
Name: categories, dtype: int64

In [16]:
#Keep only restaurants in British Columbia
restaurant = df_b[df_b['categories'].str.contains(
              'Restaurants',
              case=False, na=False)]
print("Before State {}".format(restaurant.shape))
restaurant =  restaurant[restaurant['state'].str.contains(
              'BC',
              case=False, na=False)]
print("After State {}".format(restaurant.shape))

Before State (32022, 9)
After State (4728, 9)


In [17]:
#Keep only restaurants in Vancouver
print("Before City {}".format(restaurant.shape))
restaurant = restaurant[restaurant['city'].str.contains(
              'Vancouver',
              case=False, na=False)]
print("After City {}".format(restaurant.shape))

Before City (4728, 9)
After City (2798, 9)


In [18]:
#Loading review file
size = 1000000
df_review = pd.read_json("yelp_academic_dataset_review.json", lines=True,dtype={'review_id':str,'user_id':str,'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,'funny':int,'cool':int},
                      chunksize=size)

In [19]:
# There are multiple chunks to be read
chunk_list = []
for chunk in df_review:
    # Drop columns that aren't needed
    chunk = chunk.drop(['review_id','useful','funny','cool', 'date'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk = chunk.rename(columns={'stars': 'review_stars'})
    # Inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(restaurant, chunk, on='business_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)

30836 out of 1,000,000 related reviews
26823 out of 1,000,000 related reviews
29121 out of 1,000,000 related reviews
29222 out of 1,000,000 related reviews
30709 out of 1,000,000 related reviews
28753 out of 1,000,000 related reviews
27796 out of 1,000,000 related reviews
24472 out of 1,000,000 related reviews
15904 out of 1,000,000 related reviews


In [20]:
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)
df.sample(3)

,business_id,name,city,state,stars,review_count,is_open,attributes,categories,user_id,review_stars,text
236604,DfOGJgtjaqfZyGBSKvRROA,Golden Era Burger,Vancouver,BC,4.5,7,1,None,"Food, Fast Food, Restaurants, Food Trucks, Bur...",T-vp6a7_YE9Ecb3ANp6-xw,5,Delicious burgers! I've frequented several tim...
156675,i7f9LBxucLXyCLAxGDMEmA,Rosemary Rocksalt,North Vancouver,BC,4.0,82,1,"{'RestaurantsGoodForGroups': 'False', 'Restaur...","Coffee & Tea, Sandwiches, Cafes, Bagels, Food,...",QTzj_TjMCDuYXeEMFVM_LQ,4,"Chewy, warm Montreal-style bagels in a variety..."
139737,WdywxkSYEc_nw0kqNfRZsg,Steamworks Brewing Company,Vancouver,BC,3.5,711,1,"{'HasTV': 'True', 'GoodForKids': 'False', 'Res...","Bars, Pubs, American (New), Shopping, Restaura...",0JOY3EZh_BfD_n3MvqWAEA,3,Beer tasting!\n\nThis place is HUGE!! It is de...


In [21]:
#Convert dataframe to csv
df.to_csv("yelp_reviews_restaurants.csv", index=False)
